Data Extraction

In [2]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd


load_dotenv()

API_KEY = os.getenv('NEWS_API_KEY')
RELIABLE_DOMAINS =  "kompas.com,detik.com,liputan6.com,tempo.co,cnnindonesia.com,cnbcindonesia.com"
SEARCH_QUERY = "Indonesia"
PROJECT_ID = 'indonesia-news-466813'
DESTINATION_TABLE = 'news_analysis.daily_indonesian_news'

def extract_news_data(api_key, domains, search_query='Indonesia'):
    """
    Mengambil maksimal 100 artikel berita dari domain spesifik.
    Disesuaikan untuk batasan paket gratis NewsAPI.
    """
    print(f"Memulai ekstraksi dari domain: {domains}...")
    
    URL = 'https://newsapi.org/v2/everything'
    
    # Konfigurasi parameter hanya untuk satu halaman
    query_params = {
        'q': search_query,
        'domains': domains,
        'language': 'id',
        'sortBy': 'publishedAt', # Urutkan berdasarkan yang terbaru
        'apiKey': api_key,
        'pageSize': 100  # Minta jumlah artikel maksimal
    }
    
    try:
        response = requests.get(URL, params=query_params)
        
        if response.status_code != 200:
            print(f"Gagal mengambil data. Status: {response.status_code} - Pesan: {response.json().get('message', '')}")
            return None
            
        data = response.json()
        articles = data.get('articles', [])
        
        if not articles:
            print("Tidak ada artikel yang ditemukan dari domain tersebut.")
            return None
        
        df_raw = pd.DataFrame(articles)
        # Hapus duplikat jika ada
        df_raw.drop_duplicates(subset=['title'], inplace=True, keep='first')
        
        print(f"Ekstraksi selesai. Total {len(df_raw)} artikel unik berhasil didapatkan.")
        return df_raw

    except Exception as e:
        print(f"Terjadi error pada saat ekstraksi: {e}")
        return None
    
df_raw = extract_news_data(API_KEY, RELIABLE_DOMAINS, SEARCH_QUERY)

Memulai ekstraksi dari domain: kompas.com,detik.com,liputan6.com,tempo.co,cnnindonesia.com,cnbcindonesia.com...
Ekstraksi selesai. Total 42 artikel unik berhasil didapatkan.


Data Transform

In [ ]:
# transformation.py

import pandas as pd
from textblob import TextBlob

df = df_raw.copy()

def get_sentiment(text):
    """Menganalisis dan mengkategorikan sentimen dari sebuah teks."""
    # Membuat objek TextBlob
    analysis = TextBlob(str(text))
    # Menentukan sentimen berdasarkan polaritas
    if analysis.sentiment.polarity > 0:
        return 'Positif'
    elif analysis.sentiment.polarity < 0:
        return 'Negatif'
    else:
        return 'Netral'

def clean_and_transform_data(df):
    """
    Membersihkan dan mentransformasi data mentah.
    Mengembalikan DataFrame yang siap untuk dimuat.
    """
    print("Memulai proses transformasi data...")
    
    # 1. Menangani data hilang dan duplikat
    df.dropna(subset=['title', 'content'], inplace=True)
    df.drop_duplicates(subset=['title'], inplace=True, keep='first')
    
    # 2. Konversi tipe data tanggal
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    
    # 3. Membongkar kolom 'source'
    df['source_name'] = df['source'].apply(lambda x: eval(str(x))['name'])
    
    # 4. Analisis Sentimen (Fitur Baru)
    df['sentiment'] = df['title'].apply(get_sentiment)
    
    # 5. Memilih dan menata ulang kolom
    df_cleaned = df[['publishedAt', 'source_name', 'author', 'title', 'sentiment', 'description', 'url', 'content']]
    
    print("Transformasi data selesai.")
    return df_cleaned

# Menyimpan data bersih ke file CSV
df = clean_and_transform_data(df)
df.to_csv('30 Hari Terakhir Berita Indonesia.csv', index=False, encoding='utf-8-sig')
print("Data bersih telah disimpan ke news_cleaned.csv")

Memulai proses transformasi data...
Transformasi data selesai.


Load to Big Query

In [7]:
# loading.py

from google.cloud import bigquery

def load_to_bigquery(df, project_id, destination_table):
    """
    Memuat DataFrame ke tabel di Google BigQuery.
    """
    print(f"Memulai proses memuat data ke BigQuery...")
    
    # Tentukan tujuan lengkap tabel (dataset.table)
    # Contoh: destination_table = "news_analysis.daily_indonesian_news"
    
    client = bigquery.Client(project=project_id)
    
    # Konfigurasi job untuk mode 'append'
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        autodetect=True,
    )
    
    # Unggah data
    job = client.load_table_from_dataframe(
        df, destination_table, job_config=job_config
    )
    job.result()  # Menunggu proses selesai
    
    # Verifikasi
    table = client.get_table(destination_table)
    print(f"Proses Selesai. Tabel '{table.table_id}' sekarang memiliki {table.num_rows} baris.")